### Getting started


#### Create workspace
1. Create folder
2. File > Save Workspace As  

#### Install pipenv
1. conda install -c conda-forge pipenv

#### Run pipenv
1. *conda activate* to activate environment with conda installed 
2. *pipenv shell* to activate virtual environment

ps: a script can be writen to run pipenv shell
1. create a file activate.sh
2. write pipenv shell in the file and save
3. in the terminal type in chmod +x ./.../activate.sh (this is to allow this script to be run as executable in linux)
4. run the file ./.../activate.sh

#### Install Django in virtual environent
1. *pipenv shell* to activate virtual environment
2. *pipenv install django==3.0.8 --python 3.8* to install Django
3. Pipfile will be created
3. *pip freeze* to check what packages are insatlled on this virtual environment



### Creating project

#### Starting Django Project
1. type in *django-admin startproject name_of_project .* in the terminal
2. folder *name_of_project* will be created
3. to run the django server, type in *python manage.py runserver* in the terminal

#### Migrate 
1. *python manage.py migrate*, taking Django code, adjust database to fit Django code

#### Create admin account
1. *python manage.py createsuperuser*
2. enter username, email, password

#### Acces Django admin page
1. *python manage.py runserver port_number*
2. Add /admin to the Django url
3. login using the username and password previously created

### Creating app component (emails)

1. *python manage.py startapp emails*, where emails is the app component name
2. A folder named *emails* will be created
3. models.py is where database is defined
4. add the code below in the models.py file

In [ ]:
class EmailEntry(models.Model):
    email=models.EmailField()

5. Update database about the app/model created
6. go back to TECK_landing folder > settings.py
7. look for INSTALLED_APPS
8. add the emails app into the list
9. *python manage.py makemigrations*, preparing to change the database
10. *python manage.py migrate*, change the database
11. go to emails > admin.py, import EmailEntry model by entering the code below

In [ ]:
from .models import EmailEntry

admin.site.register(EmailEntry)

### Making changes to models

1. Add additional info for email, date added, timestamp when the email is added

In [ ]:
updated=models.DateTimeField(auto_now=True) # set when saved
timestamp=models.DateTimeField(auto_now_add=True) # set when added

2. Add field for name

In [ ]:
name= models.CharField(max_length=120, blank=True)

3. Add field for bio

In [ ]:
bio=models.TextField(blank=True)

### Django and python shell

#### Start Django managed shell
1. *python manage.py shell*

#### Access global variables/config in settings.py 
1. *from django.conf import settings*, this will work on every project
2. var_name=getattr(settings, 'var_name','default') OR var_name=settings.var_name
3. when do you store things in the settings.py file? credentials etc

#### Import a model
1. *from \<appname\>.models import \<KlassName\>*
   *from emails.models import EmailEntry*

#### Get a single stored item
1. *EmailEntry.objects.get(id=1)*
2. *EmailEntry.objects.get(email="test@yahoo.com")*

#### List all stored items of a Model
1. *EmailEntry.objects.all()

#### Filter all stored items of a model
1. *EmailEntry.objects.filter(email='test@yahoo.com')*

#### Create a new stoerd item (instance) of a model
1. *EmailEntry.objects.create(email='test1@abc.com')* 
OR
2. obj=EmailEntry()
   obj.email='test2@abc.com'
   obj.save()
   
#### Update a new stored item (instance) of a model

1. obj=EmailEntry.objects.get(id=1)
   obj.name='Benson'
   obj.save()

#### Delete a new stored item (instance) of a model

1. obj=EmailEntry.objects.get(id=2)
   obj.delete()
   
#### CRUD (Create, Retrieve, Update, Delete)

### MODEL -> VIEW -> TEMPLATE

In [ ]:
from django.http import HttpResponse
from .emails import EmailEntry

def email_entry_get_view(request, *args, **kwargs):
    #get a single item stored in the database
    obj=EmailEntry.objects.get(id=1)
    return HttpResponse(f"<h1>Hello World {obj.email}</h1>")

def email_entry_list_view():
    return

def email_entry_create_view():
    return

def email_entry_update_view():
    return


#### Routing to your view
Static routing

In [ ]:
#from emails import views

# more preferably
from emails.views import email_entry_get_view

urlpatterns=[
#   path('email/1/',views.email_entry_get_view)
    path('email/1/',email_entry_get_view)
]

Dynamic routing

In [ ]:
# dynamic routing 
# urls.py
from emails.views import email_entry_get_view

urlpatterns=[
    path('email/<int:id>',email_entry_get_view)
]

# views.py

def email_entry_get_view(request, id, *args, **kwargs): #include id as input from url
    obj=EmailEntry.objects.get(id=id) # id=id from url
    return HttpResponse(f"<h1>Hello World {obj.email}</h1>")

#### Handling exceptions


In [ ]:
# views.py
from django.http import HttpResponse,Http404

def email_entry_get_view(request, id, *args, **kwargs): 
    try:
        obj=EmailEntry.objects.get(id=id) 
    except:
        raise Http404
        
    return HttpResponse(f"<h1>Hello World {obj.email}</h1>")

In [ ]:
# Be more specific when declaring the exception
# views.py
from django.http import HttpResponse,Http404

def email_entry_get_view(request, id, *args, **kwargs): 
    try:
        obj=EmailEntry.objects.get(id=id) 
    except EmailEntry.DoesNotExist: #specific exception
        raise Http404
        
    return HttpResponse(f"<h1>Hello World {obj.email}</h1>")

#### Different ways to route the URL


In [ ]:
# url.py
from django.urls import path,re_path

urlpatterns=[
    path('email/<int:id>',email_entry_get_view)
    re_path(r'email/(?P<id>\d+)/$',email_entry_get_view) # regular expression
    url(r'email/(?P<id>\d+)/$',email_entry_get_view) # regular expression
]

#### Setting up templates

1. create a directory called templates under TECK-landing
2. in settings.py search for TEMPLATES, add the templates folder path

In [ ]:
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [os.path.join(BASE_DIR,'templates')], #add folder path in DIRS
***

3. create a new file in templates folder get.html
4. add the following to the file

In [ ]:
<!doctype html>
<html>
    <body>
        <h1>{{email}}</h1> # double brackets
    </body>
</html>

In [ ]:
# views.py
def email_entry_get_view(request, id, *args, **kwargs): 
    try:
        obj=EmailEntry.objects.get(id=id) 
    except EmailEntry.DoesNotExist: #specific exception
        raise Http404
        
    return render(request, "get.html",{"email":obj.email},{"object":obj})

### Static files
1. create folder static/img under TECK-landing, this is for hosting static file
2. create folder my_CDN, this is Content Delivery Network for serving files to website 
3. go to the settings.py file, add the following code

In [ ]:
# Local static files (not being served ever)
# telling Django where the local static files are stored
STATICFILES_DIRS=[       
    os.path.join(BASE_DIR, 'static')
]

# CDN - Content Delivery Network - "SERVING" dir
STATIC_ROOT = os.path.join(BASE_DIR, 'my_CDN','static')

4. From bash, run *python manage.py collectstatic*
5. Fom urls.py add the following

In [ ]:
from django.conf import settings
from django.conf.urls.static import static

if settings.DEBUG: # denotes in DEVELOPMENT not Production
    urlpatterns+=static(settings.STATIC_URL,
    document_root=settings.STATIC_ROOT)


In [ ]:
# in get.html file

<!doctype html>
<html>
    <body>
        <img src='/static/img/coding.jpg' alt='image'/>
        <h1>{{object.email}}</h1>
    </body>
</html>

In [ ]:
# alternative way
{% load static %}
<!doctype html>
<html>
    <body>
        <img src={% static '/img/coding.jpg' %} alt='image'/>
        <h1>{{object.email}}</h1>
    </body>
</html>

#### How to run static files?
1. self-hosted
- static file server (as shown above for development)
- whitenoise (on Heroku; not recommended)
- AWS S3/Google Cloud Storage  
2. CDN
- user PUBLIC cdn files
- create our own CDN
  - AWS CloudFront
  - Google Coud CDN
  - CloudFlare
  - Stackpath
  
  


#### Model Forms
1. under emails folder create file forms.py

In [ ]:
from django import forms
from .models import EmailEntry

class EmailEntryForm(forms.ModelForm):
    class Meta:
        model=EmailEntry
        fields=['email']